# COVID-19 - Ministerio de Sanidad

In [1]:
import requests
import pandas as pd
import ezodf
import datetime as dt
import glob
import matplotlib.pyplot as plt
import time
from pprint import pprint
from pandas_ods_reader import read_ods

import matplotlib.dates as mdates 
import matplotlib.ticker as tkr

**Fuente: Ministerio de Sanidad del Gobierno de España** 
<br> https://www.sanidad.gob.es/home.htm
<br> https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm

**MUST INSTALL ezodf** - 
https://pypi.org/project/ezodf/#description
    

Use "pip install ezodf" (conda install ezodf won't work)

**MUST INSTALL pandas-ods-reader** - https://pypi.org/project/pandas-ods-reader/

Use "pip install pandas-ods-reader --user" (conda install ezodf won't work)

### Obtaining the Data

In [2]:
# Lista de fechas hasta hoy
curr_date = pd.to_datetime('today').date()

fechitas = pd.bdate_range(start='1/1/2021', end=curr_date).date
lista_fechas = list(fechitas)

print(str(lista_fechas[-1]).replace("-", ""))

20220207


In [3]:
# Fechas sin datos
# 20210101
# 20210105
# 20210106
# 20210319
# 20210402

lista_fechas.remove(dt.date(2021,1,1))
lista_fechas.remove(dt.date(2021,1,5))
lista_fechas.remove(dt.date(2021,1,6))
lista_fechas.remove(dt.date(2021,3,19))
lista_fechas.remove(dt.date(2021,4,2))
#lista_fechas.remove(dt.date(2021,5,28))

#lista_fechas.append(dt.date(2021,4,3))
lista_fechas[:] = sorted(lista_fechas)

lista_fechas = lista_fechas[:]
lista_fechas

[datetime.date(2021, 1, 4),
 datetime.date(2021, 1, 7),
 datetime.date(2021, 1, 8),
 datetime.date(2021, 1, 11),
 datetime.date(2021, 1, 12),
 datetime.date(2021, 1, 13),
 datetime.date(2021, 1, 14),
 datetime.date(2021, 1, 15),
 datetime.date(2021, 1, 18),
 datetime.date(2021, 1, 19),
 datetime.date(2021, 1, 20),
 datetime.date(2021, 1, 21),
 datetime.date(2021, 1, 22),
 datetime.date(2021, 1, 25),
 datetime.date(2021, 1, 26),
 datetime.date(2021, 1, 27),
 datetime.date(2021, 1, 28),
 datetime.date(2021, 1, 29),
 datetime.date(2021, 2, 1),
 datetime.date(2021, 2, 2),
 datetime.date(2021, 2, 3),
 datetime.date(2021, 2, 4),
 datetime.date(2021, 2, 5),
 datetime.date(2021, 2, 8),
 datetime.date(2021, 2, 9),
 datetime.date(2021, 2, 10),
 datetime.date(2021, 2, 11),
 datetime.date(2021, 2, 12),
 datetime.date(2021, 2, 15),
 datetime.date(2021, 2, 16),
 datetime.date(2021, 2, 17),
 datetime.date(2021, 2, 18),
 datetime.date(2021, 2, 19),
 datetime.date(2021, 2, 22),
 datetime.date(2021, 2, 

In [4]:
def update_data():
    i = 0
    while (i < len(lista_fechas)):
        fecha = str(lista_fechas[i]).replace("-", "")
        url = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_" + fecha + ".ods"
        resp = requests.get(url)
        with open(fecha + ".ods", "wb") as my_file:
            my_file.write(resp.content)
        resp.close()
        i = i + 1
    print("Updated all data succesfully!!")

In [5]:
# Run this to download all the data.
# It may take a while

#update_data()

### Last Week

In [6]:
# Get index of the most recent Monday
k = len(lista_fechas)-1
while(k > 0):
    random_day = k
    day_of_week = lista_fechas[random_day].weekday()

    if (day_of_week == 0):        
        index_last_monday = random_day
        break
    else:
        index_last_monday = random_day - day_of_week    
    k = k - 1

In [7]:
# index in our list of dates of the most recent Monday
index_last_monday

281

In [8]:
# comprobar si el indice obtenido es el correcto
lista_fechas[101]

datetime.date(2021, 5, 31)

In [9]:
# lista de fechas de esta última semana
last_week = lista_fechas[index_last_monday-1:]
last_week

[datetime.date(2022, 2, 4), datetime.date(2022, 2, 7)]

In [10]:
# we create empty lists for the information we will obtain
week_dosis_admin = []
week_people_one_dose = []
week_people_completed_dose = []

In [11]:
# open each file and get info
# filename error ??
dia = 0
while (dia < len(last_week)):
    
    fecha = str(last_week[dia]).replace("-", "")
    filename = str(fecha) + '.ods'
    doc = ezodf.opendoc(filename)
    

    # convert the first sheet to a pandas.DataFrame
    sheet = doc.sheets[0]
    df_dict = {}
    for i, row in enumerate(sheet.rows()):
        # row is a list of cells
        # assume the header is on the first row
        if i == 0:
            # columns as lists in a dictionary
            df_dict = {cell.value:[] for cell in row}
            # create index for the column headers
            col_index = {j:cell.value for j, cell in enumerate(row)}
            continue
        for j, cell in enumerate(row):
            # use header instead of column index
            df_dict[col_index[j]].append(cell.value)
            
    
    jeje = df_dict.get('Dosis administradas (2)')
    res = list(filter(None, jeje))
        
    dosis_admin = res[-1]
    week_dosis_admin.append(dosis_admin)
    
    jiji = df_dict.get('Nº Personas con al menos 1 dosis')
    res2 = list(filter(None, jiji))
    persons1dosis = res2[-1]
    week_people_one_dose.append(persons1dosis)
    
    jojo = df_dict.get('Nº Personas vacunadas\n(pauta completada)')
    res3 = list(filter(None, jojo))
    personscompleta = res3[-1]
    week_people_completed_dose.append(personscompleta)
    
    hundredpercent = 47450795
    
    porcentaje_personscompleta = (personscompleta * 100) / hundredpercent
    porcentaje_personscompleta
    
    porcentaje_persons1dosis = (persons1dosis * 100) / hundredpercent
    porcentaje_persons1dosis
    
    dia = dia + 1

In [12]:
# check info obtained
week_dosis_admin
week_people_one_dose
week_people_completed_dose

[38316906.0, 38327784.0]

### Report from last Week

In [13]:
tam = len(week_dosis_admin)
index = 0
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
dosis_semana = 0
week_daily_incr = []

print('*Informe Semanal COVID*')
print('\033[4m' + 'Dosis administradas esta semana:' + '\033[0m')

while(index < tam-1):
    daily_incr = week_dosis_admin[index+1] - week_dosis_admin[index]
    week_daily_incr.append(daily_incr)
    res = '{:,.0f}'.format(daily_incr)
    dosis_semana = dosis_semana + daily_incr
    
    print(weekDays[index] + '   \t=\t' + res) #only with df of a week
    index = index + 1



personas_incompleta = porcentaje_persons1dosis
personas_completa = porcentaje_personscompleta

print('\nEsta semana un total de \t' + '{:,.0f}'.format(dosis_semana) + ' dosis')
print('\n% población con al menos una dosis: \t' + '{:.2f}'.format(personas_incompleta) + '%')
print('% población pauta completa: \t\t' + '{:.2f}'.format(personas_completa) + '%')

*Informe Semanal COVID*
Dosis administradas esta semana:
Monday   	=	228,002

Esta semana un total de 	228,002 dosis

% población con al menos una dosis: 	86.32%
% población pauta completa: 		80.77%


### Progress Bar

In [14]:
progressbar = '['
indice = 0
mini_pcomp = personas_completa/10
mini_pcomp = round(mini_pcomp)
while(indice < 10):
    if(mini_pcomp <= indice):
        progressbar = progressbar + '□'
    else:
        progressbar = progressbar + '■'
    indice = indice + 1
progressbar = progressbar + '] ' + '{:.2f}'.format(personas_completa) + '% complete'
print(progressbar)

[■■■■■■■■□□] 80.77% complete
